In [1]:
import pandas as pd
import numpy as np
import geopandas as gdf
from shapely.geometry import Point
import pathlib

In [4]:
curr_dir = pathlib.Path().resolve()
print(curr_dir)

/home/inductive-anks/yield-prediction/Yield-Prediction/Code Files/Centroid-SOI


In [12]:
home_dir = curr_dir.parent.parent

In [15]:
df_centroid = gdf.read_file(home_dir.as_posix() + '/data/Centroid_File/coc_2008_2020_v1.shp')

In [16]:
df_SOI = gdf.read_file(home_dir.as_posix() + '/data/All-Inida-SOI/All-India-Villages-SOI.shp')

In [17]:
df_centroid.head()

,state,year,zone,dtnmcoc,slnococ,tehnmcoc,vilnmcoc,sdnm_11,geometry
0,Bihar,2008-11,1,Samastipur,1,Samastipur,Fatehpur,Samastipur,POINT (85.74868 25.8446)
1,Bihar,2008-11,1,Samastipur,2,Samastipur,Muktapur,Samastipur,POINT (85.75294 25.8497)
2,Bihar,2008-11,1,Samastipur,3,Dalsingsarai,Banghara,Dalsinghsarai,POINT (85.81624 25.66342)
3,Bihar,2008-11,1,Madhubani,4,Madhubani,Saurath,Madhubani,POINT (86.05196 26.3594)
4,Bihar,2008-11,1,Madhubani,4,Madhubani,Saurath,Madhubani,POINT (86.06495 26.38378)


In [18]:
df_SOI.head()

,NAME,dtnmcoc,state,tehnmcoc,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [19]:
df_SOI.rename(columns={'NAME':'VILLAGE'}, inplace=True)

## Applying the Within Fuction for Merging 

In [9]:
value_counts = df_centroid['geometry'].value_counts()

In [10]:
filtered_values = value_counts[value_counts == 1].index

In [11]:
df_filtered_centroid = df_centroid[df_centroid['geometry'].isin(filtered_values)]

In [12]:
df_filtered_centroid.shape

(3472, 9)

In [13]:
df_centroid.shape

(3773, 9)

In [28]:
type(df_filtered_centroid)

geopandas.geodataframe.GeoDataFrame

In [30]:
type(df_SOI)

geopandas.geodataframe.GeoDataFrame

In [41]:
df_SOI['villnamecoc'] = None

for i, centroid_row in df_filtered_centroid.iterrows():
    point = centroid_row['geometry']
    villnamecoc = centroid_row['vilnmcoc']
    
    if point is not None:
        for j, polygon_row in df_SOI.iterrows():
            polygon = polygon_row['geometry']
            
            if polygon is not None and point.within(polygon):
                df_SOI.at[j, 'villnamecoc'] = villnamecoc

In [31]:
import geopandas as gpd

# Convert your DataFrames to GeoDataFrames if they aren't already
df_SOI = gpd.GeoDataFrame(df_SOI, geometry='geometry')
df_filtered_centroid = gpd.GeoDataFrame(df_filtered_centroid, geometry='geometry')

# Ensure both GeoDataFrames use the same coordinate reference system (CRS)
df_filtered_centroid = df_filtered_centroid.to_crs(df_SOI.crs)

# Perform the spatial join
joined_df = gpd.sjoin(df_filtered_centroid, df_SOI, how='left', predicate='within')

# Extract and merge the village names
df_SOI['villnamecoc'] = joined_df['vilnmcoc']

# If you need to keep only the rows from df_SOI with the matching villages:
df_SOI = df_SOI.merge(joined_df[['index_right', 'vilnmcoc']], left_index=True, right_on='index_right', how='left')


In [34]:
df_filtered_centroid.to_file('filtered_centroid', driver = 'ESRI Shapefile')

In [33]:
df_SOI.sample(30)

,VILLAGE,dtnmcoc,state,tehnmcoc,geometry,villnamecoc,index_right,vilnmcoc
NaN,sonwadi,aurang>b>d,maharashtra,kannad,"POLYGON ((75.1985 20.49245, 75.20114 20.49215,...",NaN,278516.0,NaN
NaN,mund dattaji tryambak,amar>vati,maharashtra,dhamangaon,"POLYGON ((78.22782 20.83963, 78.22765 20.8392,...",NaN,291645.0,NaN
NaN,mavadi,shimoga,karnataka,tirthahalli,"POLYGON ((75.14644 13.79063, 75.14735 13.79172...",NaN,206471.0,NaN
NaN,wadiramrao,w>sh|m,maharashtra,malegaon,"POLYGON ((77.10602 20.37365, 77.10757 20.37358...",NaN,273911.0,NaN
NaN,bijaypur,sidhi,madhya pradesh,gopadbanas,"POLYGON ((81.89636 24.38571, 81.89709 24.38579...",NaN,427115.0,NaN
NaN,chandi nagar pt v,cachar,assam,katigora,"POLYGON ((92.51585 24.93931, 92.51765 24.9393,...",NaN,59614.0,NaN
NaN,kukura chuwa,sibsagar,assam,amguri,"POLYGON ((94.62704 26.90717, 94.6277 26.90726,...",NaN,45335.0,NaN
NaN,bahorwa,supaul,bihar,kishanpur,"POLYGON ((86.77533 26.19914, 86.77539 26.19882...",NaN,112488.0,NaN
NaN,deokali,fatehpur,uttarpradesh,khaga,"POLYGON ((81.12855 25.96846, 81.12846 25.96835...",NaN,516222.0,NaN
NaN,barka nuawan,buxar,bihar,buxar,"POLYGON ((83.96123 25.55214, 83.96392 25.55218...",NaN,125764.0,NaN
